In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from bs4 import BeautifulSoup

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Ping a website and return HTML of the website.

In [2]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

Reading the source code for a given web page and creating a BeautifulSoup object with the BeautifulSoup function. We extract all data and put into dataframe.

In [3]:
soup = BeautifulSoup(website_url)
table = soup.find("table",{"class":"wikitable sortable"})
table = table.find("tbody")
table = table.find_all("tr")
table_header = table[0]
table_content = table[1:]

header = [col.contents[0] for col in table_header.find_all("th")]
header = [text.replace("\n","").strip() for text in header]

value_list = []
for i in range(len(table_content)):
    content = [col.contents[0] for col in table_content[i].find_all("td")]
    content = [text.replace("\n","").strip() for text in content]
    value_list.append(content)

In [4]:
data = pd.DataFrame(value_list,columns=header)
data = data[data['Borough']!="Not assigned"]
data = data.reset_index(drop=True)
data.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
print(data.shape)

(103, 3)


Add column Latitude and Longtitude to data dataframe

In [7]:
geo = pd.read_csv("Geospatial_Coordinates.csv")
data = data.merge(geo,on='Postal Code',how='left')
data.head()

,Postal Code,Borough,Neighborhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M3A,North York,Parkwoods,43.753259,-79.329656,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,43.662301,-79.389494
